In [1]:
from datasets import load_dataset
import pandas as pd

In [2]:
echr = load_dataset("ecthr_cases",  "violation-prediction")

In [3]:
train_dataset, val_dataset, test_dataset = echr['train'], echr['validation'], echr['test']
train_dataset, val_dataset, test_dataset = [dataset.map( lambda examples: {"text": "\n".join(examples["facts"])}) for dataset in [train_dataset, val_dataset, test_dataset]]
train_dataset, val_dataset, test_dataset = [dataset.map( lambda examples: {'labels' :list(1 if examples['labels'][i] else 0 for i in range(len(examples['labels'])))}, batched=True) for dataset in [train_dataset, val_dataset, test_dataset]]

In [4]:
train_df = pd.DataFrame(data={'text': train_dataset['text'], 'result': train_dataset['labels'], 'labels': echr['train']['labels'], 'facts': echr['train']['facts']})

In [5]:
train_df['result'] = train_df.result.apply(lambda x: 'VIOLATED' if x==1 else 'NON_VIOLATED')

In [6]:
train_df[train_df['result'] == 'VIOLATED'].count()

text      8238
result    8238
labels    8238
facts     8238
dtype: int64

In [7]:
val_df = pd.DataFrame(data={'text': val_dataset['text'], 'result': val_dataset['labels'], 'labels': echr['validation']['labels'],  'facts': echr['validation']['facts']})
val_df['result'] = val_df.result.apply(lambda x: 'VIOLATED' if x==1 else 'NON_VIOLATED')

In [8]:
val_df[val_df['result'] == 'VIOLATED'].count()

text      852
result    852
labels    852
facts     852
dtype: int64

In [9]:
test_df = pd.DataFrame(data={'text': test_dataset['text'], 'result': test_dataset['labels'], 'labels': echr['test']['labels'],  'facts': echr['test']['facts']})
test_df['result'] = test_df.result.apply(lambda x: 'VIOLATED' if x==1 else 'NON_VIOLATED')

In [10]:
test_df[test_df['result'] == 'VIOLATED'].count()

text      865
result    865
labels    865
facts     865
dtype: int64

In [11]:
train_df['words_count'] = train_df['text'].str.split().apply(lambda x: len(x))
test_df['words_count'] = test_df['text'].str.split().apply(lambda x: len(x))
val_df['words_count'] = val_df['text'].str.split().apply(lambda x: len(x))

In [12]:
total_df = pd.concat([train_df, test_df, val_df], axis=0)

In [13]:
non_violated_df = total_df[total_df['result'] == 'NON_VIOLATED'].sample(800)

In [14]:
non_violated_df

,text,result,labels,facts,words_count
3182,"6. The applicant was born in 1969 and, before...",NON_VIOLATED,[],"[6. The applicant was born in 1969 and, befor...",394
8810,5. The applicant was born in 1982 and lives i...,NON_VIOLATED,[],[5. The applicant was born in 1982 and lives ...,3445
528,5. The applicant was born in 1977 and lives i...,NON_VIOLATED,[],[5. The applicant was born in 1977 and lives ...,2611
43,9. The applicant made the following allegatio...,NON_VIOLATED,[],[9. The applicant made the following allegati...,3608
966,"7. The applicants, Olavi Puolitaival and Esko...",NON_VIOLATED,[],"[7. The applicants, Olavi Puolitaival and Esk...",1030
...,...,...,...,...,...
1210,11. The applicant company is an airline chart...,NON_VIOLATED,[],[11. The applicant company is an airline char...,10359
7382,5. The applicant was born in 1973 and lives i...,NON_VIOLATED,[],[5. The applicant was born in 1973 and lives ...,1998
7154,5. The applicant was born in 1945 and lives i...,NON_VIOLATED,[],[5. The applicant was born in 1945 and lives ...,729
7736,"6. The first applicant, Ms Zita Paliukienė, w...",NON_VIOLATED,[],"[6. The first applicant, Ms Zita Paliukienė, ...",2987


In [15]:
violated_df = total_df[total_df['result'] == 'VIOLATED'].sample(800)

In [16]:
total_df.words_count.median()

1032.5

In [17]:
total_df.words_count.describe()

count    11000.000000
mean      1662.080455
std       1967.597964
min         69.000000
25%        505.000000
50%       1032.500000
75%       2069.250000
max      35416.000000
Name: words_count, dtype: float64

In [18]:
train_df['words_count'].describe()

count     9000.000000
mean      1619.237111
std       1995.117127
min         69.000000
25%        480.750000
50%        984.000000
75%       2002.250000
max      35416.000000
Name: words_count, dtype: float64

In [19]:
train_df[train_df['result'] == 'VIOLATED'].count()

text           8238
result         8238
labels         8238
facts          8238
words_count    8238
dtype: int64

In [20]:
train_df[train_df['result'] == 'NON_VIOLATED'].count()

text           762
result         762
labels         762
facts          762
words_count    762
dtype: int64

In [21]:
total_df[total_df['result'] == 'NON_VIOLATED'].count()

text           1045
result         1045
labels         1045
facts          1045
words_count    1045
dtype: int64

In [22]:
total_df.shape

(11000, 5)

In [23]:
train_df.shape

(9000, 5)

In [24]:
test_df.shape

(1000, 5)

In [25]:
val_df.shape

(1000, 5)

In [28]:
total_df.loc[0, 'facts'].to_csv('facts.csv')

In [30]:
total_df.loc[0, 'text'].to_csv('text.csv')